In [1]:
import tensorflow as tf
from timeit import repeat
import numpy as np
import tensorboard
!rm -rf ./logs/

tf.config.list_physical_devices('GPU') 

2024-03-13 16:58:55.183035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 16:58:55.183133: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 16:58:55.183156: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 16:58:55.190726: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 16:58:57.765100: I tensor

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
@tf.function
def neural_net(x: tf.Tensor, w1: tf.Tensor, w2: tf.Tensor, b1: tf.Tensor, b2: tf.Tensor):
    assert x.dtype == tf.float32
    assert w1.dtype == tf.float32
    assert w2.dtype == tf.float32
    assert b1.dtype == tf.float32
    assert b2.dtype == tf.float32
    assert x.shape == (10,) or x.shape == (1, 10)
    
    x = tf.reshape(x, [1, 10])
    layer1 = x @ w1 + b1 # matmul (@), bo to siec gleboka    
    layer2 = tf.sigmoid(layer1) @ w2 + b2
    
    y = tf.sigmoid(layer2)
    return y

def neural_net_undecorated(x: tf.Tensor, w1: tf.Tensor, w2: tf.Tensor, b1: tf.Tensor, b2: tf.Tensor):
    assert x.dtype == tf.float32
    assert w1.dtype == tf.float32
    assert w2.dtype == tf.float32
    assert b1.dtype == tf.float32
    assert b2.dtype == tf.float32
    assert x.shape == (10,) or x.shape == (1, 10)

    x = tf.reshape(x, [1, 10])
    layer1 = x @ w1 + b1 # matmul (@), bo to siec gleboka    
    layer2 = tf.sigmoid(layer1) @ w2 + b2
    
    y = tf.sigmoid(layer2)
    return y

In [3]:
logdir = 'logs/func/lab3'
writer = tf.summary.create_file_writer(logdir)

2024-03-13 16:58:57.805673: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 16:58:57.805808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 16:58:57.805858: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 16:58:57.928374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 16:58:57.928443: I tensorflow/compile

In [4]:
w1 = tf.constant([[ 1.4334732, -1.5244598 , 1.139654 , 2.723477, 2.372128, -1.8221221],
                  [ 1.6633688, -1.3922757, 2.0349483, 1.6314147, 1.6997916, -1.719175],
                  [ 1.6464833, -1.6136154, 1.6790704, 2.1913328, 1.7154503, -2.122219],
                  [ 2.2029521, -2.2169485, 1.1411709, 1.7363839, 1.9620435, -1.990284],
                  [ 1.864349, -1.9724554, 1.282788, 1.3895663 , 1.2881863, -1.3681948],
                  [ 0.4421571, 0.24537054, 0.49080196, -0.0939824, 0.36308903, -0.32526237],
                  [-1.6102886, 1.7532632, -1.3683709, -1.2728035, -1.8335032, 1.6637068],
                  [-1.0453694, 0.95990705, -1.913037, -1.637573 , -1.8312218, 1.9757035],
                  [-2.6982157, 1.5073962, -2.243781, -2.7327728 , -2.5648139, 1.9095569],
                  [-2.0628226, 2.3980575, -1.3550557, -2.1798916 , -2.1485612, 2.2912557]], tf.float32)

b1 = tf.constant([-1.6460503, 1.5486399, -1.5155386, -1.6247352, -1.2638505, 1.5515162], tf.float32)
w2 = tf.constant([[ 2.220895 ], [-2.903738 ], [ 1.7675139], [ 2.3042984], [ 2.6292808], [-2.763858 ]], tf.float32)
b2 = tf.constant([-1.1827456], tf.float32)
x1 = tf.constant([0.85, 0.86, 0.76, 0.73, 0.95, 0.5, 0.5, 0.5, 0.5, 0.5], tf.float32)
x2 = tf.constant([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.67, 0.87], tf.float32)

tf.summary.trace_on(graph=True, profiler=True)
print(x1, "\n--->\n", neural_net(x1, w1, w2, b1, b2))
with writer.as_default():
    tf.summary.trace_export(
        name="neural_net",
        step=0,
        profiler_outdir=logdir)
print(x2, "\n--->\n", neural_net(x2, w1, w2, b1, b2))

Instructions for updating:
use `tf.profiler.experimental.start` instead.


2024-03-13 16:58:58.034456: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-03-13 16:58:58.034507: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-03-13 16:58:58.034609: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1694] Profiler found 1 GPUs


tf.Tensor([0.85 0.86 0.76 0.73 0.95 0.5  0.5  0.5  0.5  0.5 ], shape=(10,), dtype=float32) 
--->
 tf.Tensor([[0.9977336]], shape=(1, 1), dtype=float32)
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
tf.Tensor([0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.67 0.87], shape=(10,), dtype=float32) 
--->
 tf.Tensor([[0.00667112]], shape=(1, 1), dtype=float32)


2024-03-13 16:58:58.595389: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-03-13 16:58:58.596178: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1828] CUPTI activity buffer flushed
2024-03-13 16:58:58.600917: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_collector.cc:541]  GpuTracer has collected 19 callback api events and 17 activity events. 
2024-03-13 16:58:58.601356: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


In [5]:
%load_ext tensorboard
%tensorboard --logdir logs

In [9]:
#start = time.time()
#for _ in range(3):
#    neural_net(x1, w1, w2, b1, b2)
#    neural_net(x2, w1, w2, b1, b2)
#elapsed = time.time() - start
neural_net(x1, w1, w2, b1, b2)
neural_net(x2, w1, w2, b1, b2)
print("Czas z dekoratorem (x1):",    np.mean(repeat(lambda: neural_net(x1, w1, w2, b1, b2), number=1, repeat=1000)),
      "\nCzas z dekoratorem (x2):",  np.mean(repeat(lambda: neural_net(x2, w1, w2, b1, b2), number=1, repeat=1000)))
#start = time.time()
#for _ in range(3):
#    neural_net_undecorated(x1, w1, w2, b1, b2)
#    neural_net_undecorated(x2, w1, w2, b1, b2)
#elapsed = time.time() - start
print("\nCzas bez dekoratora (x1):",  np.mean(repeat(lambda: neural_net_undecorated(x1, w1, w2, b1, b2), number=1, repeat=1000)),
      "\nCzas bez dekoratora (x2):",  np.mean(repeat(lambda: neural_net_undecorated(x2, w1, w2, b1, b2), number=1, repeat=1000)))

Czas z dekoratorem (x1): 0.0016411504749985397 
Czas z dekoratorem (x2): 0.0010549790709974331

Czas bez dekoratora (x1): 0.0009853551579935812 
Czas bez dekoratora (x2): 0.0009218312649800282


Dlaczego??

> Unfortunately, tf.function is not always faster for small computations. The reason is quite simple: calling an empty tf.function is more expensive than calling an empty Python function. This is especially true when the computations involve scalars, when the speed gained inside the function is too small to recover the cost of calling it.  ~mdanatg